In [3]:
#https://teddylee777.github.io/langchain/langchain-tutorial-04/
#https://wikidocs.net/234282
#https://medium.com/bosphorusiss/part-1-3-chat-with-a-csv-langchain-openai-5f342d0523a5

In [1]:
import os
import openai
import pandas as pd
import json
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [4]:
# OpenAI API 키 설정
openai.api_key = "개인키입력"
os.environ['OPENAI_API_KEY'] = "개인키입력"

In [13]:
def text_generator(prompt, model="gpt-4o", temperature=0):
    messages = [
        {"role": "system", "content": "질문에 오류 혹은 잘못된 정보가 있는지 확인하고, 있다면 이것을 지적하고 수정한다."},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    answer = response.choices[0].message.content
    return answer

def get_arg_answer(query, verbose=False):
    prompt = f"""
    당신은 데이터 프레임에서 쿼리한 데이터에 대한 질의 응답 봇입니다. 
    특히, 국가 정보 데이터베이스를 가지고 있는 전문가입니다.
    
    df 열과 열 설명은 다음과 같습니다: 
        국가코드: 각 국가를 식별하기 위한 고유한 코드입니다.
        국가한글명: 각 국가의 한글 표기명입니다.
        국가영문명: 각 국가의 영문 표기명입니다.
        국가대륙코드: 각 국가가 속한 대륙을 나타내는 코드로 대륙명을 나타냅니다.
        
    앞으로 문장이 들어오면 국가코드, 국가한글명, 국가영문명, 국가대륙코드와 관련된 추가 질문을 10개 생성해주세요.
    예시를 제공해드립니다. 예시 외의 질문들을 추가로 작성해줘야 합니다.
    문장의 구분은 '\n'로 진행해야 합니다.
    문장들을 구분하는데 있어서 "1."와 같이 숫자가 있다면 제외해주세요.
    ex) 1. 러시아의 국가한글명은 무엇인가요? -> 러시아의 국가한글명은 무엇인가요?
    
    output은 꼭 아래의 예시 템플릿을 따라주세요.
    
    예시
    input : 국가코드, 국가한글명, 국가영어명, 국가대륙코드에 대한 csv 파일
    output : 아르메니아의 국가코드는 무엇인가요?\n아제르바이잔의 국가영문명은 무엇인가요?\n방글라데시는 어느 대륙에 속해있나요?\n"BN"은 어느나라의 국가코드인가요?\n부탄의 국가코드, 국가대륙코드는 각각 무엇인가요?

    질문은 output에 속하는 내용만 포함하면 됩니다.
    또한, 아래와 같은 형식은 -> 다음 형식으로 수정부탁드립니다.
    ex) \"IN\"은 어느나라의 국가코드인가요? -> 'IN'은 어느나라 국가코드인가요?
    
    이제 당신이 생성해야할 글을 제공해드리겠습니다.
    {query}
    """
    if verbose:
        print(f'Prompt: {prompt}')
    answer = text_generator(prompt)
    return answer.split("\n")

def get_gen_answer(questions, csv_path, verbose=False):
    answers = []
    for question in questions:
        # CSV 파일을 데이터프레임으로 로드
        df = pd.read_csv(csv_path, encoding='cp949')
    
        prompt = f"""
        당신은 데이터 프레임에서 쿼리한 데이터에 대한 질의 응답 봇입니다. 
        특히, 국가 정보 데이터베이스를 가지고 있는 전문가입니다.
        앞으로 문장이 들어오면 국가코드, 국가한글명, 국가영문명, 국가대륙코드와 관련된 질문에 전문가로서 답변을 해주세요.
        빠르게 답변할 필요 없이 생각하고 답변을 출력해주시면 됩니다.
        관련 정보를 최대한 활용하여 답변을 출력하면 됩니다.

        df 열과 열 설명은 다음과 같습니다: 
        국가코드: 각 국가를 식별하기 위한 고유한 코드입니다.
        국가한글명: 각 국가의 한글 표기명입니다.
        국가영문명: 각 국가의 영문 표기명입니다.
        국가대륙코드: 각 국가가 속한 대륙을 나타내는 코드로 대륙명을 나타냅니다.

        출력물의 최대 문자 길이는 300자로 제한하겠습니다.
        답변을 최대 문자 길이 내로 축약해서 전달해주면 됩니다.

        답변은 예시와 같이 제공하면 됩니다.
        input : 아르메니아의 국가코드는 무엇인가요?\n아제르바이잔의 국가영문명은 무엇인가요?\n방글라데시는 어느 대륙에 속해있나요?\n'BN'은 어느나라의 국가코드인가요?\n부탄의 국가코드, 국가대륙코드는 각각 무엇인가요?
        output : 아르메니아의 국가코드는 'AM'입니다.\n아제르바이잔의 국가영문명은 'AZERBAIJAN'입니다.\n방글라데시의 국가대륙코드는 '아시아'로 아시아에 속해있습니다.\n'BN'은 브루나이의 국가코드입니다.\n부탄의 국가코드는 'BT', 국가대륙코드는 '아시아'입니다.

        답변은 output에 속하는 내용만 포함하면 됩니다.
            
        이제 당신이 답변해야할 글을 제공해드리겠습니다.
        {question}
        """
        if verbose:
            print(f'Prompt for question "{question}": {prompt}')
        answer = text_generator(prompt)
        answers.append(answer)
    return answers

def generate_qa_from_csv(csv_path, verbose=False):
    # CSV 파일을 데이터프레임으로 로드
    df = pd.read_csv(csv_path, encoding='cp949')

    # 데이터프레임을 텍스트로 변환
    text = df.to_string(index=False)

    # 질문과 답변 생성
    questions = get_arg_answer(text, verbose=verbose)
    answers = get_gen_answer(questions, csv_path, verbose=verbose)

    # JSON 형식으로 변환
    qa_pairs = [{"QUESTION": q.strip(), "ANSWER": a.strip()} for q, a in zip(questions, answers)]

    return qa_pairs

csv_path = "D:/[24]ICT_Practice/practice_file/code.csv"
qa_pairs = generate_qa_from_csv(csv_path)

print(json.dumps(qa_pairs, ensure_ascii=False, indent=2))

[
  {
    "QUESTION": "아르메니아의 국가코드는 무엇인가요?",
    "ANSWER": "아르메니아의 국가코드는 'AM'입니다."
  },
  {
    "QUESTION": "아제르바이잔의 국가영문명은 무엇인가요?",
    "ANSWER": "아제르바이잔의 국가영문명은 'AZERBAIJAN'입니다."
  },
  {
    "QUESTION": "방글라데시는 어느 대륙에 속해있나요?",
    "ANSWER": "방글라데시의 국가대륙코드는 'AS'로 아시아에 속해있습니다."
  },
  {
    "QUESTION": "'BN'은 어느나라 국가코드인가요?",
    "ANSWER": "'BN'은 브루나이의 국가코드입니다."
  },
  {
    "QUESTION": "부탄의 국가코드, 국가대륙코드는 각각 무엇인가요?",
    "ANSWER": "부탄의 국가코드는 'BT', 국가대륙코드는 '아시아'입니다."
  },
  {
    "QUESTION": "중국의 국가영문명은 무엇인가요?",
    "ANSWER": "중국의 국가영문명은 'CHINA'입니다."
  },
  {
    "QUESTION": "건지섬의 국가한글명은 무엇인가요?",
    "ANSWER": "건지섬의 국가한글명은 '건지섬'입니다."
  },
  {
    "QUESTION": "홍콩의 국가코드는 무엇인가요?",
    "ANSWER": "홍콩의 국가코드는 'HK'입니다."
  },
  {
    "QUESTION": "인도네시아의 국가대륙코드는 무엇인가요?",
    "ANSWER": "인도네시아의 국가대륙코드는 '아시아'입니다."
  },
  {
    "QUESTION": "'IN'은 어느나라 국가코드인가요?",
    "ANSWER": "'IN'은 인도의 국가코드입니다."
  }
]
